In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import sys
import csv
import os
from sklearn import metrics
from dateutil.parser import parse
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import lightgbm as lgb
%matplotlib inline

In [2]:
import numpy as np, pandas as pd, os
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

In [3]:
data_version_input = "01-10-2020"

In [4]:
df_train = pd.read_csv(f"../../data/kalapa/{data_version_input}/train.csv")
df_test = pd.read_csv(f"../../data/kalapa/{data_version_input}/test.csv")

/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_transform = pd.concat([df_train.iloc[:,2:], df_test.iloc[:,1:]], axis = 0)
data_type = {
    "cat_num":["Field_22","Field_13", "Field_41", "Field_42","Field_50", "Field_51", "Field_53",\
               "Field_73",  "partner0_A", "partner0_B", "partner0_C", "partner0_D", "partner0_E",\
               "partner0_F", "partner0_G", "partner0_H", "partner1_A", "partner1_C", "partner1_G", "partner1_H",\
               "partner2_A", "partner2_C", "partner2_D", "partner2_F", "partner2_H","partner3_A", "partner3_D",\
               "partner3_E", "partner4_L", "partner5_A", "partner5_D", "partner5_G"],
    
    "cat_str" : ["Field_3","Field_4", "Field_12","Field_36", "Field_38", "Field_54", "Field_62","Field_65", \
                 "Field_66", "info_social_sex", "brief", "gioiTinh"], 
    
    "date" : ["Field_1", "Field_2", "Field_5", "Field_6", "Field_7", "Field_8", "Field_9", "Field_11",\
             "Field_15", "Field_25", "Field_32", "Field_33", "Field_35", "Field_40","Field_43", "Field_44", \
              "F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
             "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate", "ngaySinh"],
    
    "text" : ["Field_18","currentLocationCity", "homeTownCity", "Field_48", "Field_49",\
             "currentLocationCountry", 'homeTownState','data.basic_info.locale', "currentLocationName", \
              'homeTownName', "homeTownCountry", "maCv", "currentLocationState", "Field_61", "Field_56",],
    
    "drop" : ["Field_45", "Field_34","Field_68", "diaChi", "Field_46", "Field_55"]
}

df_transform = df_transform.drop(data_type["drop"], axis = 1)
df_transform = df_transform.drop(data_type["text"], axis = 1)
df_transform = df_transform.drop(data_type["date"], axis = 1)
for feature in  data_type["cat_num"]:
    df_transform[feature] = df_transform[feature].fillna("Nan")
    df_transform = pd.get_dummies(df_transform, columns=[feature])
for feature in  data_type["cat_str"]:
    df_transform[feature] = df_transform[feature].fillna("Nan")
    df_transform = pd.get_dummies(df_transform, columns=[feature])
"""    
for feature in data_type["date"]:
    df_transform[feature] = df_transform[feature].map(lambda x: float(parse(str(x)).timestamp()) if type(x) is str else x)
    df_transform[feature] = df_transform[feature].fillna(df_transform[feature].mean())
"""
train_dev = df_transform.iloc[:len(df_train),:]
train_dev = train_dev.fillna(train_dev.mean())

test = df_transform.iloc[len(df_train):,:]
test = test.fillna(test.mean())

scaler = MinMaxScaler()
scaler.fit(train_dev)
X_transform = scaler.transform(train_dev)
X_transform = pd.DataFrame(data = X_transform)

test_transform = scaler.transform(test)

test = pd.DataFrame(data = test_transform)
train_dev = pd.concat([df_train.label, X_transform], axis = 1)
test = pd.concat([df_test.id, test], axis = 1)
train_dev.reset_index(drop=True,inplace=True)

In [6]:
"""
pca = PCA(0.90)
sel = pca.fit(train_dev.iloc[:,1:])
train_variance = sel.transform(train_dev.iloc[:,1:])
test_variance = sel.transform(test.iloc[:,1:])
train_variance  = pd.DataFrame(data = train_variance)
test_variance  = pd.DataFrame(data = test_variance)
train = pd.concat([df_train.id, df_train.label, train_variance], axis = 1)
test = pd.concat([df_test.id, test_variance], axis = 1)
"""

'\npca = PCA(0.90)\nsel = pca.fit(train_dev.iloc[:,1:])\ntrain_variance = sel.transform(train_dev.iloc[:,1:])\ntest_variance = sel.transform(test.iloc[:,1:])\ntrain_variance  = pd.DataFrame(data = train_variance)\ntest_variance  = pd.DataFrame(data = test_variance)\ntrain = pd.concat([df_train.id, df_train.label, train_variance], axis = 1)\ntest = pd.concat([df_test.id, test_variance], axis = 1)\n'

In [7]:
"""
sel = VarianceThreshold(threshold=0.1).fit(train_dev.iloc[:,1:])
train_variance = sel.transform(train_dev.iloc[:,1:])
test_variance = sel.transform(test.iloc[:,1:])
train_variance  = pd.DataFrame(data = train_variance)
test_variance  = pd.DataFrame(data = test_variance)
train = pd.concat([df_train.id, df_train.label, train_variance], axis = 1)
test = pd.concat([df_test.id, test_variance], axis = 1)
"""

'\nsel = VarianceThreshold(threshold=0.1).fit(train_dev.iloc[:,1:])\ntrain_variance = sel.transform(train_dev.iloc[:,1:])\ntest_variance = sel.transform(test.iloc[:,1:])\ntrain_variance  = pd.DataFrame(data = train_variance)\ntest_variance  = pd.DataFrame(data = test_variance)\ntrain = pd.concat([df_train.id, df_train.label, train_variance], axis = 1)\ntest = pd.concat([df_test.id, test_variance], axis = 1)\n'

In [8]:
train = pd.concat([df_train.id, train_dev], axis = 1)

In [9]:
train

,id,label,0,1,2,3,4,5,6,7,...,300,301,302,303,304,305,306,307,308,309
0,0,1,0.200000,0.000000,0.142906,0.750000,0.034483,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,0,0.219308,0.038902,0.120912,0.747555,0.067829,0.011264,0.011115,0.090948,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,0,0.200000,0.000000,0.167785,0.750000,0.021552,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,0,0.219308,0.038902,0.120912,0.747555,0.067829,0.011264,0.011115,0.090948,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,1,0.219308,0.038902,0.120912,0.747555,0.067829,0.011264,0.011115,0.090948,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53025,53025,0,0.200000,0.145688,0.050000,0.750000,0.010776,0.000000,0.000000,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
53026,53026,0,0.219308,0.038902,0.120912,0.747555,0.067829,0.011264,0.011115,0.090948,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
53027,53027,0,0.200000,0.000000,0.134732,0.750000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
53028,53028,0,0.219308,0.038902,0.120912,0.747555,0.067829,0.011264,0.011115,0.090948,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [10]:
test

,id,0,1,2,3,4,5,6,7,8,...,300,301,302,303,304,305,306,307,308,309
0,53030,0.200000,0.000000,0.050000,0.750000,0.092672,0.000000,0.000000,0.166667,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,53031,0.200000,0.000000,0.035000,0.750000,0.051724,0.000000,0.000000,0.166667,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,53032,0.220199,0.043991,0.122980,0.748262,0.070501,0.009317,0.009205,0.091841,0.025276,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,53033,0.200000,0.000000,0.050000,0.750000,0.000000,0.000000,0.000000,0.166667,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,53034,0.220199,0.043991,0.122980,0.748262,0.070501,0.009317,0.009205,0.091841,0.025276,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20376,73406,0.220199,0.043991,0.122980,0.748262,0.070501,0.009317,0.009205,0.091841,0.025276,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20377,73407,0.220199,0.043991,0.122980,0.748262,0.070501,0.009317,0.009205,0.091841,0.025276,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20378,73408,0.200000,0.602564,0.206800,0.750000,0.370690,0.000000,0.000000,0.333333,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
20379,73409,0.220199,0.043991,0.122980,0.748262,0.070501,0.009317,0.009205,0.091841,0.025276,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
data_version = "07-10-2020"

In [12]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
train.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)